# 1. Setting up a confusion matrix

In [12]:
#Load the data

import csv
import pandas as pd
import numpy as np
from sklearn import metrics


with open('data_test.csv', 'r') as file:
    df = csv.reader(file)
    for row in df:
        print(row)
        
df = pd.read_csv('data_test.csv')
        
#Create the confusion matrix
# Importing the dependancies

y_pred = df[df.columns[-1]]
# Actual values
y_act = df[df.columns[2]]

# Printing the confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.

print(y_pred)
# Printing the precision and recall, among other metrics

print(y_act)
#confusion matrix
print(metrics.confusion_matrix(y_act, y_pred))

#other model metrics
print(metrics.classification_report(y_act, y_pred))


['\ufeffx1', 'x2', 'y_1', 'y_predicted']
['test 1', 'temp 1', '0', '0']
['test 2', 'temp 2', '0', '1']
['test 2', 'temp 3', '1', '1']
0    0
1    1
2    1
Name: y_predicted, dtype: int64
0    0
1    0
2    1
Name: y_1, dtype: int64
[[1 1]
 [0 1]]
